In [1]:
import random
import numpy as np
from ParticleClass import hello as hell
from FuncionesGeneradorasListas import FuncionesListas as fl
import matplotlib.pyplot as plt
from FGeneradora import FuncionGeneradora as fg

In [ ]:
#hola

In [2]:
%load_ext Cython

Lo que hice: Corrí una simulación con los parametros que se muestran abajo y le construí el histograma.

In [15]:
%%time
%%cython 
#Se importan las cosas necesarias 
from ParticleClass import hello as hell
import timeit
from FuncionesGeneradorasListas import FuncionesListas as fl
import matplotlib.pyplot as plt
from FGeneradora import FuncionGeneradora as fg

#Parametros simulación
cdef float v1 = 30  #Velocidad mínima
cdef float v2 = 150  #Velocidad máxima
cdef float dt = 0.01 #tamaño del paso
cdef float Lx = 200 #Tamñaño de la caja 
cdef float Ly = 200 
cdef int n = 150 #número de partículas
cdef float m = 1 #masa 
cdef float r = 2 #radio
cdef int nt  = 10000 #número de pasos
cdef int j,k,i




def simulate(float v1, float v2, float dt, float Lx, float Ly, int n, float m, float r, int nt):
    par = fg.funcion_generadora(Lx,Ly,v1,v2,m,r,n)
    lista_posiciones = fl.listas_para_llenar_posiciones(n)
    lista_velocidades = fl.listas_para_llenar_componentes_velocidad(n)
    for j in range(nt):

        for k in range(n):
            lista_posiciones[k][0].append(par[k].posicion[0])  #Se añade la posición en x de cada partícula
            lista_posiciones[k][1].append(par[k].posicion[1])  #Se añade la posicion en y de cada partícula
            lista_velocidades[k][0].append(par[k].velocidad[0]) #Se añade la velocidad en x de cada partícula
            lista_velocidades[k][1].append(par[k].velocidad[1]) #Se añade la velocidad en y de cada partícula
        for p in par:     
            for i in range(par.index(p)+1,n): #Revisamos las colisiones entre partículas
                if  p.posicion[0] == par[i].posicion[0] and p.posicion[1] == par[i].posicion[1]:
                    continue 

                elif p.ver_colision_pp(par[i])  :
                    #print("CHOQUE")
                    p.resolver_colision_particula(par[i])
        for p in par: 
            if p.ver_colision_muro(Lx,Ly): #Revisamos las condiciones de choque con la caja
                p.resolver_colision_muro(Lx,Ly)
            elif p.ver_colision_esquina(Lx,Ly): #Revisamos las condiciones de choque con las esquinas
                p.resolver_colision_esquina()
            p.paso_dt(dt)
    return lista_posiciones , lista_velocidades
        

Wall time: 6.78 s


En estás celdas generé una lista con todas las magnitudes que se presentan en cada paso dado.

Ahora importo scipy para calcular las frecuencias relativas de la. De ahora en adelante me guíe por lo que está 
aquí: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.relfreq.html 

In [33]:
A=simulate(0,100,0.01,100,100,2,1,1,3)[1]#Definimos una lista A, que contiene la historia de las velcidades de cada partícula

In [58]:
#Esta es la idea principal que luego formalizamos en la función temperatura, imprimimos resultados para comparar
print(A)
l=[]
for n in range(2):
    
    v_x=A[n][0][-1]
    v_y=A[n][1][-1]
    v_cuadrado=(v_x)**2+(v_y)**2
    l.append(v_cuadrado)
    
print(l)

[[[46.286903381347656, 46.286903381347656, 46.286903381347656], [71.81390380859375, 71.81390380859375, 71.81390380859375]], [[89.75392150878906, 89.75392150878906, 89.75392150878906], [41.990169525146484, 41.990169525146484, 41.990169525146484]]]
[7299.714204864169, 9818.940762956408]


In [55]:
def temperatura(n,lista_v,m):#La función recibe el número de partículas, la lista de velcidades que va a provenir de la simulación
    l=[]                     #y la masa de las partículas(esta será igual siempre y la podremos sacar de los parámetros)
    for i in range(n):
    
      v_x=lista_v[i][0][-1]
      v_y=lista_v[i][1][-1]
      v_cuadrado=(v_x)**2+(v_y)**2
      l.append(v_cuadrado)
    Temperatura=0.5*m*(sum(l)/float(len(l)))#Se calcula el promedio de la energía cinética en todo el gas para el último paso
    return Temperatura


In [56]:
temperatura(2,A,1)#Prueba usando la lista A, efectivamente, da lo que tiene que dar haciendo el cálculo a mano

4279.663741955144

In [59]:
def Area(lx,ly):
    return lx*ly
